Data Collection Part 1: Cleaning Up and Creating Fire Data

In [5]:
import pandas as pd
import random
import numpy as np
import requests
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url = 'https://raw.githubusercontent.com/AlexLiu2233/Forest-Fire-Predictor/main/data/H_FIRE_PNT.csv'
df = pd.read_csv(url, index_col=0)
df

C:\Users\liujp\AppData\Local\Temp\ipykernel_6984\3262594084.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, index_col=0)


,FIRE_YEAR,IGN_DATE,FIRE_CAUSE,FIRELABEL,FRCNTR,ZONE,FIRE_ID,FIRE_TYPE,GEO_DESC,LATITUDE,LONGITUDE,SIZE_HA,FCODE,SHAPE,OBJECTID,X,Y
FIRE_NO,,,,,,,,,,,,,,,,,
K30894,2001,2.001082e+13,Person,2001-K30894,5,3,500894,Nuisance Fire,Nielson Beach #1,51.0167,-119.0167,NaN,JA70003000,NaN,11402317,-119.016667,51.016667
V70397,2010,2.010071e+13,Person,2010-V70397,2,7,200397,Fire,Extension Road,49.1102,-123.9395,0.009,JA70003000,NaN,11402318,-123.939484,49.110249
K40261,2004,2.004070e+13,Lightning,2004-K40261,5,4,500261,Fire,Fintry,50.1537,-119.5490,0.009,JA70003000,NaN,11402319,-119.548950,50.153733
R50229,2019,2.019051e+13,Person,2019-R50229,3,5,300229,Fire,.6km south of Arbor FSR on Hwy 16,55.8015,-128.8971,0.009,JA70003000,NaN,11402320,-128.897117,55.801450
V81229,2009,2.009080e+13,Person,2009-V81229,2,8,201229,Nuisance Fire,Cumberland Dump,49.6422,-125.0558,0.000,JA70003000,NaN,11402321,-125.055783,49.642200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N10266,1999,1.999090e+07,Person,1999-N10266,6,1,266,Fire,FORT STEELE,49.4923,-115.4473,0.009,JA70003000,NaN,11579248,-115.447333,49.492333
N10267,1999,1.999081e+07,Lightning,1999-N10267,6,1,267,Fire,CAVEN CREEK,49.2630,-115.3325,0.200,JA70003000,NaN,11579249,-115.332500,49.263000
N10269,1999,1.999091e+07,Person,1999-N10269,6,1,269,Fire,Hidden Valley,49.4748,-115.7871,0.100,JA70003000,NaN,11579250,-115.787067,49.474767


In [6]:
#Dropping features we dont need
df.drop(columns=['FIRE_YEAR', 'FIRELABEL', 'FRCNTR', 'ZONE', 'OBJECTID', 'FIRE_ID', 'GEO_DESC', 'FCODE', 'SHAPE', 'OBJECTID', 'X', 'Y'], inplace=True, errors='ignore')

#Making it so that data rows have valid SIZE_HA (fire size in hectares) and IGN_DATE (date of one row of data)
df['IGN_DATE'] = df['IGN_DATE'].div(1000000)
df = df[df['SIZE_HA'].notnull()]
df = df[df['IGN_DATE'].notnull()]

#Use data after 1970
df = df[df['IGN_DATE'] >= 1970]

#Group data that has one common feature: There is no fire
df_nofire = df[df['SIZE_HA'] == 0]
df_nofire = df_nofire.head(2500)

#Group data that has one common feature: There is fire (We define "there is fire" as there exists fire with SIZE_HA larger than 1.9)
df_yesfire = df[df['SIZE_HA'] > 1.9]

In [7]:
total_nofire_list = []
#Create data for no fire (this is fine because the chances that a fire happening at a random location given random date is pretty small)
for i in range(2500):
  one_entry = []
  lat = random.uniform(48, 60)
  lon = -random.uniform(130, 115)
  date_yyyy = random.randint(1970, 2020)
  date_mm = random.randint(1,12)

  if (date_mm <= 3 or date_mm == 5 or date_mm == 7 or date_mm == 8 or date_mm == 10 or date_mm == 12):
    if (date_mm == 2):
      date_dd = int(random.randint(1,28))
    else:
      date_dd = int(random.randint(1,31))
  else:
    date_dd = int(random.randint(1,30))
  
  one_entry.append(date_yyyy*10000+date_mm*100+date_dd)
  one_entry.append("No Cause")
  one_entry.append("No Fire")
  one_entry.append(lat)
  one_entry.append(lon)
  one_entry.append(0.0)
  total_nofire_list.append(one_entry)

df2 = pd.DataFrame(total_nofire_list, columns=df.columns)
df_nofire = pd.concat([df_nofire, df2], ignore_index=True)

In [8]:
df_nofire.to_csv('data_nofire.csv')
df_yesfire.to_csv('data_yesfire.csv')

Data Collection Part 2: Getting Climate Data

In [9]:
import requests
import pandas as pd

In [10]:
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url_yes = 'https://raw.githubusercontent.com/AlexLiu2233/Forest-Fire-Predictor/main/data/data_yesfire.csv'
df_yesfire = pd.read_csv(url_yes, index_col=0)
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url_no = 'https://raw.githubusercontent.com/AlexLiu2233/Forest-Fire-Predictor/main/data/data_nofire.csv'
df_nofire = pd.read_csv(url_no, index_col=0)

In [11]:
df_yesfire

,IGN_DATE,FIRE_CAUSE,FIRE_TYPE,LATITUDE,LONGITUDE,SIZE_HA
FIRE_NO,,,,,,
C30085,2.015052e+07,Lightning,Fire,52.2711,-121.5118,3.6
K71366,2.017073e+07,Person,Fire,50.3299,-121.3953,30.0
N20709,2.000081e+07,Person,Fire,49.8731,-115.6792,3.5
G90279,2.010071e+07,Lightning,Fire,58.8575,-121.9688,4.7
G40080,2.000051e+07,Person,Fire,53.9200,-124.0817,25.0
...,...,...,...,...,...,...
G70024,2.010041e+07,Person,Fire,56.0713,-121.8325,2.8
G41084,2.019072e+07,Lightning,Fire,54.0024,-124.6282,2.0
C20014,2.016041e+07,Person,Fire,51.5929,-123.0197,17.0


In [12]:
df_nofire

,IGN_DATE,FIRE_CAUSE,FIRE_TYPE,LATITUDE,LONGITUDE,SIZE_HA
V81229,2.009080e+07,Person,Nuisance Fire,49.642200,-125.055800,0.0
V10028,2.013052e+07,Person,Nuisance Fire,49.121100,-121.591700,0.0
V10055,2.011052e+07,Person,Nuisance Fire,49.077200,-121.707800,0.0
V70761,2.008101e+07,Person,Nuisance Fire,49.341200,-124.420900,0.0
V60046,2.004043e+07,Unknown,Nuisance Fire,48.783300,-123.950000,0.0
...,...,...,...,...,...,...
2495,1.971071e+07,No Cause,No Fire,54.524433,-121.838602,0.0
2496,1.999021e+07,No Cause,No Fire,48.405481,-116.337460,0.0
2497,1.994050e+07,No Cause,No Fire,51.126953,-127.393534,0.0
2498,1.977022e+07,No Cause,No Fire,59.242536,-125.893874,0.0


In [13]:
def split(word):
  return [char for char in word]

#To get historical climate data given date, longitude and latitude from visual-crossing rapidapi (Used to add additional features for training/test data)
def get_climate_data(ign_date, latitude, longitude):

  url = "https://visual-crossing-weather.p.rapidapi.com/history"

  ign_date_list = split(str(ign_date))
  yyyy = ''.join(ign_date_list[0:4])
  mm = ''.join(ign_date_list[4:6])
  dd = ''.join(ign_date_list[6:8])
  start_dt = f'{yyyy}-{mm}-{dd}T00:00:00'

  coords = f'{latitude}, {longitude}'

  querystring = {"startDateTime":start_dt,"aggregateHours":"24","location":coords,"endDateTime":start_dt,"unitGroup":"us","dayStartTime":"8:00:00","contentType":"csv","dayEndTime":"17:00:00","shortColumnNames":"0"}

  headers = {
      'x-rapidapi-key': "5416e60fd3msh25313be4cf91e39p1db2f2jsn7acc7269aacf",
      'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com"
      }

  response = requests.request("GET", url, headers=headers, params=querystring)
  response_list = response.text.split(",")

  return response_list
  # f.write(response.text)

#To get forecasted climate data given longitude and latitude from visual-crossing rapidapi (Used in the finished website to get necessary features, given a user input, to feed into finished model)
def get_future_climate_data(latitude, longitude):
  url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
  coords = f'{latitude}, {longitude}'
  querystring = {"location":coords,"aggregateHours":"24","shortColumnNames":"0","unitGroup":"us","contentType":"csv"}

  headers = {
      'x-rapidapi-key': "5416e60fd3msh25313be4cf91e39p1db2f2jsn7acc7269aacf",
      'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com"
      }

  response = requests.request("GET", url, headers=headers, params=querystring)
  response_list = response.text.split(",")

  return response_list

In [14]:
get_climate_data(20030705.09, 49.0707, -121.7133)

['Address',
 'Date time',
 'Minimum Temperature',
 'Maximum Temperature',
 'Temperature',
 'Dew Point',
 'Relative Humidity',
 'Heat Index',
 'Wind Speed',
 'Wind Gust',
 'Wind Direction',
 'Wind Chill',
 'Precipitation',
 'Precipitation Cover',
 'Snow Depth',
 'Visibility',
 'Cloud Cover',
 'Sea Level Pressure',
 'Weather Type',
 'Latitude',
 'Longitude',
 'Resolved Address',
 'Name',
 'Info',
 'Conditions\n"49.0707',
 ' -121.7133"',
 '"07/05/2003"',
 '58.9',
 '71.4',
 '64.2',
 '57.9',
 '81.15',
 '',
 '6.8',
 '',
 '232.7',
 '',
 '0.02',
 '10.0',
 '0.0',
 '9.4',
 '74.1',
 '1019.6',
 '"Mist',
 ' Rain Showers',
 ' Heavy Rain',
 ' Light Rain',
 ' Sky Coverage Increasing"',
 '49.0707',
 '-121.7133',
 '"49.0707',
 ' -121.7133"',
 '"49.0707',
 ' -121.7133"',
 '""',
 '"Rain',
 ' Partially cloudy"\n']

In [15]:
get_future_climate_data(1,32)

['Address',
 'Date time',
 'Latitude',
 'Longitude',
 'Resolved Address',
 'Name',
 'Wind Direction',
 'Minimum Temperature',
 'Maximum Temperature',
 'Temperature',
 'Wind Speed',
 'Cloud Cover',
 'Heat Index',
 'Chance Precipitation (%)',
 'Precipitation',
 'Sea Level Pressure',
 'Snow Depth',
 'Snow',
 'Relative Humidity',
 'Wind Gust',
 'Wind Chill',
 'Conditions\n"1',
 ' 32"',
 '"04/29/2024"',
 '1.0',
 '32.0',
 '"1',
 ' 32"',
 '"1',
 ' 32"',
 '149.7',
 '64.8',
 '83.4',
 '71.9',
 '4.4',
 '42.4',
 '84.6',
 '90.3',
 '0.02',
 '1011.7',
 '0.0',
 '0.0',
 '82.3',
 '16.3',
 '',
 '"Rain',
 ' Partially cloudy"\n"1',
 ' 32"',
 '"04/30/2024"',
 '1.0',
 '32.0',
 '"1',
 ' 32"',
 '"1',
 ' 32"',
 '186.4',
 '64.1',
 '83.9',
 '71.5',
 '3.5',
 '43.2',
 '85.9',
 '83.9',
 '0.13',
 '1010.8',
 '0.0',
 '0.0',
 '85.0',
 '10.7',
 '',
 '"Rain',
 ' Partially cloudy"\n"1',
 ' 32"',
 '"05/01/2024"',
 '1.0',
 '32.0',
 '"1',
 ' 32"',
 '"1',
 ' 32"',
 '171.9',
 '64.1',
 '79.4',
 '70.2',
 '3.0',
 '52.7',
 '',
 '87

In [16]:
tomorrow_climate_data = get_future_climate_data(0,0)[0:46]
tomorrow_climate_data

['Address',
 'Date time',
 'Latitude',
 'Longitude',
 'Resolved Address',
 'Name',
 'Wind Direction',
 'Minimum Temperature',
 'Maximum Temperature',
 'Temperature',
 'Wind Speed',
 'Cloud Cover',
 'Heat Index',
 'Chance Precipitation (%)',
 'Precipitation',
 'Sea Level Pressure',
 'Snow Depth',
 'Snow',
 'Relative Humidity',
 'Wind Gust',
 'Wind Chill',
 'Conditions\n"0',
 ' 0"',
 '"04/29/2024"',
 '0.0',
 '0.0',
 '"0',
 ' 0"',
 '"0',
 ' 0"',
 '159.4',
 '83.0',
 '84.5',
 '84.1',
 '9.4',
 '96.2',
 '94.7',
 '96.8',
 '0.24',
 '1009.4',
 '',
 '',
 '80.1',
 '11.6',
 '',
 '"Rain']

In [17]:
#Reformat fire data to include additional climate data obtained from rapidAPI (For the purpose of training the model)
def get_climate_df(df):
  total_df = []
  date = []
  min_tmp = []
  max_tmp = []
  avg_tmp = []
  dew_point = []
  relative_humidity = []
  heat_index = []
  wind_speed = []
  wind_gust = []
  wind_dir = []
  wind_chill = []
  precipitation = []
  visibility = []
  cloud_cover = []
  sea_level_pressure = []
  weather_type = []
  print("the lenght of the df passed into this function is", len(df))
  
  for i in range (len(df)): 
    response_list = get_climate_data(df['IGN_DATE'][i], df['LATITUDE'][i], df['LONGITUDE'][i])
    
    if (len(response_list) < 22):
      response_list = ['N/A'] * 52
    date.append(response_list[26])  #date, to ensure we are obtaining the right data
    min_tmp.append(response_list[27]) #minimum temperature
    max_tmp.append(response_list[28]) #max
    avg_tmp.append(response_list[29]) #average
    dew_point.append(response_list[30]) #dew point
    relative_humidity.append(response_list[31]) #relative humidity
    heat_index.append(response_list[32]) #heat index
    wind_speed.append(response_list[33]) #wind speed
    wind_gust.append(response_list[34]) #wind gust
    wind_dir.append(response_list[35]) #wind dir
    wind_chill.append(response_list[36]) #wind chill
    precipitation.append(response_list[37]) #precipitation
    visibility.append(response_list[40]) #visibility
    cloud_cover.append(response_list[41]) #cloud cover
    sea_level_pressure.append(response_list[42]) #sea level pressure
    weather_type.append(response_list[43:]) #rest of the response

  df['minimum temperature'] = min_tmp
  df['maximum temperature'] = max_tmp
  df['average temperature'] = avg_tmp
  df['dew point'] = dew_point
  df['relative humidity'] = relative_humidity
  df['heat index'] = heat_index
  df['wind speed'] = wind_speed
  df['wind gust'] = wind_gust
  df['wind direction'] = wind_dir
  df['wind chill'] = wind_chill
  df['precipitation'] = precipitation
  df['visibility'] = visibility
  df['cloud cover'] = cloud_cover
  df['sea level pressure'] = sea_level_pressure
  df['weather_type'] = weather_type

In [ ]:
#Limit of api with free x-rapidapi-key is 500 requests, since we needed to process 10000 data, we needed to make 20 separate accounts to
#get 20 free x-rapidapi-key to process everything. This grueling process is not done only using the two lines below but also through some Excel work

#In any case, NEVER RUN THESE TWO LINES OF CODE AGAIN, which is why we are commenting it, as it will exhaust our current working key which is going to be used for the website

# get_climate_df(df_yesfire)
# get_climate_df(df_nofire)

the lenght of the df passed into this function is 4982
the lenght of the df passed into this function is 5000


In [18]:
df_yesfire.to_csv('yesfire_complete_data2.csv')

In [19]:
df_nofire.to_csv('nofire_complete_data2.csv')

Part 2.1: Manual Dataset Manipulation

We combine the two datasets above to a single .csv file. We later observed that only using the randomly generated fires labeled as size 0 produced better results. Hence, the final file used for subsequent sections consist of:
  1. 2500 entries from yesfire_complete_data2
  2. 2500 entries from nofire_complete_data2 - the *randomly* generated entries only.

Part 3: Spliting Dataset for Testing and Training

In [20]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

In [21]:
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url = 'https://raw.githubusercontent.com/AlexLiu2233/Forest-Fire-Predictor/main/data/test_data_milestone3.csv'
df_ = pd.read_csv(url, low_memory=False)
df_

,SIZE_HA,IGN_DATE,LATITUDE,LONGITUDE,minimum temperature,maximum temperature,average temperature,dew point,relative humidity,heat index,wind speed,wind gust,wind direction,wind chill,precipitation,visibility,cloud cover,sea level pressure,Unnamed: 18,Unnamed: 19
0,0.0,20140607.12,49.776300,-123.106700,51.8,72.8,62.5,50.5,67.49,NaN,10.2,20.8,193.71,NaN,0.01,633.7,0.0,1019.7,NaN,NaN
1,0.0,20130820.11,49.194400,-125.391900,50.3,73.5,62.4,49.1,64.63,NaN,7.8,NaN,271.46,NaN,0.00,99.0,5.9,1020.7,NaN,NaN
2,0.0,19920730.00,56.646412,-129.772666,46.5,71.7,60.9,49.6,70.22,NaN,11.4,NaN,188.57,NaN,0.00,19.5,45.8,1019.9,NaN,NaN
3,0.0,20170131.00,54.956750,-126.093563,-4.6,23.6,7.9,2.6,79.90,NaN,5.9,NaN,235.29,-0.3,0.06,96.2,3.1,1037.9,NaN,NaN
4,0.0,19781008.00,51.313833,-122.866839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,165.0,20140728.15,50.343400,-126.882200,53.8,66.8,59.0,55.8,89.73,NaN,12.4,20.8,111.38,NaN,0.00,32.3,22.4,1020.7,NaN,NaN
4994,2392.0,20180811.16,49.072300,-115.457100,61.5,93.8,78.7,42.3,28.50,90.1,12.2,26.0,182.75,NaN,0.00,91.8,10.4,1008.3,NaN,NaN
4995,19226.0,20180717.17,49.084600,-119.869500,69.4,96.9,83.0,44.8,27.67,92.4,9.2,25.3,199.54,NaN,0.00,628.4,0.0,1011.2,NaN,NaN
4996,4.0,20030731.16,48.355000,-123.575000,55.0,67.3,60.2,55.8,85.95,NaN,19.4,NaN,223.71,NaN,0.00,NaN,0.0,1014.2,NaN,NaN


In [22]:
def cleanup_data(df):
    # Helper function to check and convert invalid entries to NaN
    def to_nan(entry):
        # Attempt to convert any entry to a float, if not possible, return NaN
        try:
            return float(entry)
        except ValueError:
            return np.nan

    # Apply conversion to NaN for entries that are non-numeric strings across all columns
    for column in df.columns:
        df[column] = df[column].apply(to_nan)
        if df[column].dtype == float or df[column].dtype == int:
            # Compute the mean of the column, ignoring NaN values
            avg = df[column].mean(skipna=True)
            # Fill NaN values with the column's average
            df[column].fillna(avg, inplace=True)
            print(f"Average of {column}: {avg}")  # Print the average of each column

    # Adjust 'SIZE_HA' to avoid log(0) by incrementing each value by 1
    if 'SIZE_HA' in df.columns and not df['SIZE_HA'].isnull().all():
        df['SIZE_HA'] += 1

    # Drop unwanted columns, if they exist
    df.drop(columns=['Unnamed: 18', 'Unnamed: 19'], errors='ignore', inplace=True)

    # Eliminate outliers in 'SIZE_HA' if column exists and has valid data
    if 'SIZE_HA' in df.columns and not df['SIZE_HA'].isnull().all():
        q25, q75 = np.percentile(df['SIZE_HA'].dropna(), [25, 75])
        iqr = q75 - q25
        upper = q75 + 1.5 * iqr
        df = df[df['SIZE_HA'] <= upper]

    return df

# Usage of the function with a DataFrame
df_ = cleanup_data(df_)

Average of SIZE_HA: 330.48600520208083
Average of IGN_DATE: 20026124.90970588
Average of LATITUDE: 53.404423052496995
Average of LONGITUDE: -122.14423555004001
Average of minimum temperature: 40.58008083140877
Average of maximum temperature: 63.09509237875289
Average of average temperature: 52.08420900692841
Average of dew point: 37.187727006444064
Average of relative humidity: 62.418746338605736
Average of heat index: 85.18183962264152
Average of wind speed: 12.534633294528522
Average of wind gust: 27.29196633511859
Average of wind direction: 192.44023689197726
Average of wind chill: 27.72063157894737
Average of precipitation: 0.06612954545454545
Average of visibility: 76.20172189733594
Average of cloud cover: 30.33781051415367
Average of sea level pressure: 1015.3026817640047
Average of Unnamed: 18: nan
Average of Unnamed: 19: nan


C:\Users\liujp\AppData\Local\Temp\ipykernel_6984\2635257698.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(avg, inplace=True)


In [23]:
# Splitting the dataset into training and testing sets based on the 'IGN_DATE' column
# Entries with 'IGN_DATE' before 2015 are used for training
df_train = df_[df_['IGN_DATE'] < 20150000]
# Entries with 'IGN_DATE' from 2015 onwards are used for testing
df_test = df_[df_['IGN_DATE'] >= 20150000]

# Output the training dataset to check its content and structure
print(df_train)
# Output the testing dataset to check its content and structure
print(df_test)


      SIZE_HA     IGN_DATE   LATITUDE   LONGITUDE  minimum temperature  \
0         1.0  20140607.12  49.776300 -123.106700            51.800000   
1         1.0  20130820.11  49.194400 -125.391900            50.300000   
2         1.0  19920730.00  56.646412 -129.772666            46.500000   
4         1.0  19781008.00  51.313833 -122.866839            40.580081   
6         1.0  19911202.00  48.639189 -115.494061            40.580081   
...       ...          ...        ...         ...                  ...   
4988      6.1  20100621.18  56.225100 -121.163000            50.100000   
4990      8.4  20040624.16  50.793600 -121.487100            58.100000   
4992     13.0  20060723.23  51.202000 -118.841200            62.400000   
4996      5.0  20030731.16  48.355000 -123.575000            55.000000   
4997      9.0  20010515.15  52.126000 -123.264000            35.300000   

      maximum temperature  average temperature  dew point  relative humidity  \
0               72.800000      

In [24]:
import numpy as np

# Splitting the training dataset into features and labels
# Applying logarithmic transformation to the 'SIZE_HA' column for labels to normalize the distribution
sizeHA_train = np.log(df_train['SIZE_HA'].values)
# Removing the 'SIZE_HA' column from the training set to create the feature set
feats_train = df_train.drop(columns=['SIZE_HA'])

# Splitting the testing dataset into features and labels
# Applying logarithmic transformation to the 'SIZE_HA' column for labels to normalize the distribution
sizeHA_test = np.log(df_test['SIZE_HA'].values)
# Removing the 'SIZE_HA' column from the testing set to create the feature set
feats_test = df_test.drop(columns=['SIZE_HA'])


Part 4: Random Forest Model Development

In [28]:
!pip install scikit-learn==0.24.1
import datetime, re, math
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy import std
import sklearn, sklearn.tree, sklearn.ensemble, sklearn.feature_extraction, sklearn.metrics

You should consider upgrading via the 'C:\Users\liujp\.pyenv\pyenv-win\versions\3.9.7\python.exe -m pip install --upgrade pip' command.


In [30]:
# Correcting the import statement for DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# Initializing the DictVectorizer
dvr = DictVectorizer()

# Transforming the training and testing features into a dictionary format expected by DictVectorizer
enc_feats_train = dvr.fit_transform(feats_train.to_dict('records'))
enc_feats_test = dvr.transform(feats_test.to_dict('records'))

print(enc_feats_test)
print(enc_feats_train)

  (0, 0)	20170131.0
  (0, 1)	54.95675018
  (0, 2)	-126.0935632
  (0, 3)	7.9
  (0, 4)	3.1
  (0, 5)	2.6
  (0, 6)	85.18183962264152
  (0, 7)	23.6
  (0, 8)	-4.6
  (0, 9)	0.06
  (0, 10)	79.9
  (0, 11)	1037.9
  (0, 12)	96.2
  (0, 13)	-0.3
  (0, 14)	235.29
  (0, 15)	27.29196633511859
  (0, 16)	5.9
  (1, 0)	20170525.0
  (1, 1)	58.75955073
  (1, 2)	-122.2478543
  (1, 3)	63.8
  (1, 4)	49.0
  (1, 5)	35.7
  (1, 6)	85.18183962264152
  (1, 7)	73.7
  :	:
  (716, 9)	0.0
  (716, 10)	64.38
  (716, 11)	1015.0
  (716, 12)	630.8
  (716, 13)	27.72063157894737
  (716, 14)	211.88
  (716, 15)	30.0
  (716, 16)	15.1
  (717, 0)	20180502.06
  (717, 1)	50.6508
  (717, 2)	-122.4127
  (717, 3)	59.2
  (717, 4)	0.2
  (717, 5)	40.2
  (717, 6)	85.18183962264152
  (717, 7)	73.4
  (717, 8)	46.7
  (717, 9)	0.01
  (717, 10)	52.75
  (717, 11)	1016.6
  (717, 12)	198.9
  (717, 13)	43.3
  (717, 14)	162.88
  (717, 15)	34.4
  (717, 16)	14.9
  (0, 0)	20140607.12
  (0, 1)	49.7763
  (0, 2)	-123.1067
  (0, 3)	62.5
  (0, 4)	0.0
  (0, 5

In [31]:
def eval_model(model):
    # Predict outcomes using the trained model on the training dataset
    preds_train = model.predict(enc_feats_train)
    # Predict outcomes using the trained model on the testing dataset
    preds_test = model.predict(enc_feats_test)
    
    # Iterate through each prediction for the test set and print it alongside the actual label
    for i in range(len(preds_test)):
        print(f"the labels: {sizeHA_test[i]}: the prediction {preds_test[i]}")
    
    # Calculate mean absolute error, median absolute error, and root mean squared error for both datasets
    tbl = [[sklearn.metrics.mean_absolute_error(*vecs),
            sklearn.metrics.median_absolute_error(*vecs),
            math.sqrt(sklearn.metrics.mean_squared_error(*vecs))]
           for vecs in [(sizeHA_train, preds_train), (sizeHA_test, preds_test)]]
    
    # Create a DataFrame to neatly display the error metrics for training and testing datasets
    df = pd.DataFrame(tbl, columns=['MAE', 'MdAE', 'RMSE'], index=['training', 'testing'])
    
    # Return the DataFrame with rounded values for readability
    return df.T.round(2)

In [61]:
# Initialize the Random Forest regressor with specific parameters
model_rf = sklearn.ensemble.RandomForestRegressor(
    n_estimators=80,        # Number of trees in the forest
    max_samples=3406,       # Number of samples to draw from X to train each base estimator
    max_features=0.7,       # The number of features to consider when looking for the best split: 70% of features
    min_samples_leaf=4,     # The minimum number of samples required to be at a leaf node
    random_state=291,       # Controls both the randomness of the bootstrapping of the samples and the features
    n_jobs=-1               # Number of jobs to run in parallel (-1 means using all processors)
)

# Fit the Random Forest model to the training data
model_rf.fit(enc_feats_train, sizeHA_train)

# Evaluate the model's performance on both training and testing data
evaluation_results = eval_model(model_rf)
print(evaluation_results)

the labels: 0.0: the prediction 0.239977086194273
the labels: 0.0: the prediction 1.8812911752300772
the labels: 0.0: the prediction 0.2449856188785963
the labels: 0.0: the prediction 0.5330358457990254
the labels: 0.0: the prediction 1.1106028984859613
the labels: 0.0: the prediction 1.2148840896419892
the labels: 0.0: the prediction 1.8322745267787997
the labels: 0.0: the prediction 0.04143006544828498
the labels: 0.0: the prediction 1.3023289549816988
the labels: 0.0: the prediction 0.9823263073091228
the labels: 0.0: the prediction 1.0338428404674864
the labels: 0.0: the prediction 0.7674760381332187
the labels: 0.0: the prediction 0.14933955779457658
the labels: 0.0: the prediction 1.4321119200227597
the labels: 0.0: the prediction 1.012474401527372
the labels: 0.0: the prediction 0.1750600858299724
the labels: 0.0: the prediction 1.3603556447141003
the labels: 0.0: the prediction 0.8579518801871219
the labels: 0.0: the prediction 0.2567681955390858
the labels: 0.0: the prediction

Part 5: Multiple Regression Model


In [33]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import sklearn
import math
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


In [57]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer

# Initialize the DictVectorizer
dvr = DictVectorizer(sparse=True)

# Apply the DictVectorizer to the training data
enc_feats_train = dvr.fit_transform(feats_train.to_dict('records'))

# Create a pipeline that includes scaling and linear regression
model_pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),  # Appropriate for sparse matrices
    ('regressor', LinearRegression())
])

# Train the model using cross-validation and evaluate its performance
scores = cross_val_score(model_pipeline, enc_feats_train, sizeHA_train, cv=5, scoring='neg_mean_absolute_error')
print(f"Cross-validated MAE: {-scores.mean():.2f} (+/- {scores.std():.2f})")

# If further use of the model is required, fit the pipeline to the entire training dataset
model_pipeline.fit(enc_feats_train, sizeHA_train)

# Optionally, save your model to disk using joblib or pickle for later use
# from joblib import dump
# dump(model_pipeline, 'model_pipeline.joblib')

# Creating a DataFrame from the test labels to view the actual values
df_test = pd.DataFrame(sizeHA_test)

# Printing the DataFrame to show the actual sizeHA values from the test dataset
print(df_test)


Cross-validated MAE: 0.68 (+/- 0.18)
            0
0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.000000
..        ...
713  1.945910
714  1.435085
715  1.098612
716  1.931521
717  1.386294

[718 rows x 1 columns]


In [58]:
# Import necessary functions from sklearn.metrics
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error

# Define a function to evaluate the Linear Regression model
def eval_lr(model, feats_train, feats_test, sizeHA_train, sizeHA_test):
    # Predicting the target variable for training and testing sets
    preds_train = model.predict(feats_train)
    preds_test = model.predict(feats_test)

    # Calculating evaluation metrics: MAE, MdAE, and RMSE for both training and testing datasets
    metrics = [
        [mean_absolute_error(sizeHA_train, preds_train),
         median_absolute_error(sizeHA_train, preds_train),
         math.sqrt(mean_squared_error(sizeHA_train, preds_train))],
        [mean_absolute_error(sizeHA_test, preds_test),
         median_absolute_error(sizeHA_test, preds_test),
         math.sqrt(mean_squared_error(sizeHA_test, preds_test))]
    ]

    # Creating a DataFrame to display the metrics clearly
    df = pd.DataFrame(metrics, columns=['MAE', 'MdAE', 'RMSE'], index=['training', 'testing'])

    # Returning the transposed DataFrame for better readability, rounding to 2 decimal places
    return df.T.round(2)

# Computing the loss for half the data in the dataset (this should be implemented as needed outside this function)


In [59]:
eval_lr(model_pipeline, feats_train, feats_test, sizeHA_train, sizeHA_test)


,training,testing
MAE,1.10,1.44
MdAE,0.84,1.27
RMSE,1.42,1.68


Part 6: the ensemble

In [67]:
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
import math

def eval_ensemble(rf_model, pipeline_model, entries, sizeHA_test, display_ensemble_loss):
    """
    Evaluates an ensemble of a Random Forest model and a pipeline model.
    Assumes that 'entries' is a DataFrame with appropriate features.

    :param rf_model: trained Random Forest model
    :param pipeline_model: trained pipeline (includes any preprocessing and a regressor)
    :param entries: feature data as DataFrame for predictions
    :param sizeHA_test: actual target values for evaluation
    :param display_ensemble_loss: if set to True, print loss metrics, otherwise return predictions
    """
    # Predicting using both models
    rf_preds = rf_model.predict(entries)
    pipeline_preds = pipeline_model.predict(entries)
    
    # Averaging predictions from both models
    combined_preds = (rf_preds + pipeline_preds) / 2.0
    
    # Optionally print detailed predictions comparison
    # Optionally print detailed predictions comparison
    if display_ensemble_loss:
        for i in range(len(entries)):
            size_label = sizeHA_test[i] if isinstance(sizeHA_test, np.ndarray) else sizeHA_test.iloc[i]
            print(f"[{i}]: size_label: {size_label}, rf_preds: {rf_preds[i]}, pipeline_preds: {pipeline_preds[i]}, combined_preds: {combined_preds[i]}")
        
        # Calculating error metrics
        mae = mean_absolute_error(sizeHA_test, combined_preds)
        mdae = median_absolute_error(sizeHA_test, combined_preds)
        rmse = math.sqrt(mean_squared_error(sizeHA_test, combined_preds))

        # Print the metrics
        print(f"Mean Absolute Error (MAE): {mae:.2f}")
        print(f"Median Absolute Error (MdAE): {mdae:.2f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
        
        # Creating DataFrame to display metrics
        df = pd.DataFrame([[mae, mdae, rmse]], columns=['MAE', 'MdAE', 'RMSE'], index=['testing'])
        return df.T.round(2)
    else:
        return combined_preds



In [68]:
final_preds = eval_ensemble(model_rf, model_pipeline, feats_test, sizeHA_test, True)

[0]: size_label: 0.0, rf_preds: 1.3505632917547312, pipeline_preds: 3.2839805774605253, combined_preds: 2.3172719346076285
[1]: size_label: 0.0, rf_preds: 1.1705522672786253, pipeline_preds: -0.5027057805441189, combined_preds: 0.3339232433672532
[2]: size_label: 0.0, rf_preds: 0.9995037906042736, pipeline_preds: 2.197612627607427, combined_preds: 1.5985582091058501
[3]: size_label: 0.0, rf_preds: 1.6492283235441252, pipeline_preds: 2.357583654932199, combined_preds: 2.0034059892381624
[4]: size_label: 0.0, rf_preds: 1.3620539594708465, pipeline_preds: 0.9449361792717781, combined_preds: 1.1534950693713122
[5]: size_label: 0.0, rf_preds: 1.3930897602304757, pipeline_preds: 1.0416088612092267, combined_preds: 1.2173493107198512
[6]: size_label: 0.0, rf_preds: 1.5196693173217755, pipeline_preds: 0.6003661224736305, combined_preds: 1.060017719897703
[7]: size_label: 0.0, rf_preds: 0.8014401767856405, pipeline_preds: 0.8759905728621931, combined_preds: 0.8387153748239168
[8]: size_label: 0